In [1]:
import matplotlib.pyplot as plt
%matplotlib inline 
import seaborn as sns
import pandas as pd
df = pd.read_csv("datasets/2017_2018_preprocessing2.csv")

In [2]:
df.head()

,CASE_STATUS,CASE_SUBMITTED,DECISION_DATE,VISA_CLASS,EMPLOYMENT_START_DATE,EMPLOYMENT_END_DATE,EMPLOYER_NAME,JOB_TITLE,SOC_CODE,SOC_NAME,...,CHANGE_EMPLOYER,AMENDED_PETITION,FULL_TIME_POSITION,PREVAILING_WAGE,WAGE_RATE_OF_PAY_FROM,WORKSITE_STATE,wage_diff,regions,applications_per_employer,employer_success_rate
0,CERTIFIED,2015-10-06,2017-10-01,H-1B,2015-11-15,2017-11-14,MASSACHUSETTS INSTITUTE OF TECHNOLOGY,POSTDOCTORAL ASSOCIATE,15-2041,STATISTICIANS,...,0,0,Y,58032.0,58032.0,MA,0.0,northeast,362,98.6
1,CERTIFIED,2016-05-13,2017-10-01,H-1B,2016-09-01,2019-08-31,MASSACHUSETTS INSTITUTE OF TECHNOLOGY (MIT),ASSISTANT PROFESSOR,25-1031,"ARCHITECTURE TEACHERS, POSTSECONDARY",...,0,0,Y,60967.0,95600.0,MA,34633.0,northeast,50,96.0
2,CERTIFIED,2016-11-04,2017-10-01,H-1B,2016-12-01,2017-11-30,MASSACHUSETTS INSTITUTE OF TECHNOLOGY,POSTDOCTORAL ASSOCIATE,19-2012,PHYSICISTS,...,0,0,Y,50398.0,60400.0,MA,10002.0,northeast,362,98.6
3,CERTIFIED,2016-12-28,2017-10-01,H-1B,2017-02-01,2018-01-31,MASSACHUSETTS INSTITUTE OF TECHNOLOGY,RESEARCH SCIENTIST,19-2042,"GEOSCIENTISTS, EXCEPT HYDROLOGISTS AND GEOGRAP...",...,0,0,Y,61069.0,65000.0,MA,3931.0,northeast,362,98.6
4,CERTIFIED,2017-02-24,2017-10-01,H-1B,2017-06-01,2018-05-31,MASSACHUSETTS INSTITUTE OF TECHNOLOGY,POSTDOCTORAL ASSOCIATE,17-2131,MATERIALS ENGINEERS,...,0,0,Y,41392.0,49383.0,MA,7991.0,northeast,362,98.6


need to decide what to do with the dates and if to use them in the models the dates cant be passed to the models as data so new features must be created, what will have to be done is to extract the dates and convert them to date objects find the difference and get the differnce in days

In [3]:
from datetime import datetime
#testing how date object works
d1 = "2015-10-06"
d2 = "2017-10-01"
date_format = "%Y-%m-%d"
date_object1 = datetime.strptime(d1, date_format)
date_object2 = datetime.strptime(d2, date_format)
diff = date_object2.date() - date_object1.date()
diff.days

726

need to standardise the date formats some are different 2 different types year-month-day and day/month/year

In [4]:
def fix_date(row, date_name):
    try:
        date_format = "%Y-%m-%d"
        date_object1 = datetime.strptime(row[date_name], date_format)
        return row[date_name]
    except:
        date_format = "%d/%m/%Y"
        date_object1 = datetime.strptime(row[date_name], date_format)
        split_date = row[date_name].split("/")
        new_date = split_date[2] + "-" + split_date[1] + "-" + split_date[0]
        return new_date

In [5]:
def date_diff(row, date1_name, date2_name):
    date_format1 = "%Y-%m-%d"
    date_object1 = datetime.strptime(row[date1_name], date_format)
    date_object2 = datetime.strptime(row[date2_name], date_format)
    diff = date_object2.date() - date_object1.date()
    return diff.days

In [6]:
df["CASE_SUBMITTED"] = df.apply(lambda row: fix_date(row, "CASE_SUBMITTED"), axis=1)

In [7]:
df["DECISION_DATE"] = df.apply(lambda row: fix_date(row, "DECISION_DATE"), axis=1)

In [8]:
df["decision_duration"] = df.apply(lambda row: date_diff(row, "CASE_SUBMITTED", "DECISION_DATE"), axis=1)

In [9]:
df.decision_duration.max()

2214

In [10]:
df.decision_duration.min()

0

In [11]:
df["EMPLOYMENT_START_DATE"] = df.apply(lambda row: fix_date(row, "EMPLOYMENT_START_DATE"), axis=1)
df["EMPLOYMENT_END_DATE"] = df.apply(lambda row: fix_date(row, "EMPLOYMENT_END_DATE"), axis=1)

In [12]:
df["employment_duration"] = df.apply(lambda row: date_diff(row, "EMPLOYMENT_START_DATE",
                                                           "EMPLOYMENT_END_DATE"), axis=1)

In [13]:
df.head()

,CASE_STATUS,CASE_SUBMITTED,DECISION_DATE,VISA_CLASS,EMPLOYMENT_START_DATE,EMPLOYMENT_END_DATE,EMPLOYER_NAME,JOB_TITLE,SOC_CODE,SOC_NAME,...,FULL_TIME_POSITION,PREVAILING_WAGE,WAGE_RATE_OF_PAY_FROM,WORKSITE_STATE,wage_diff,regions,applications_per_employer,employer_success_rate,decision_duration,employment_duration
0,CERTIFIED,2015-10-06,2017-10-01,H-1B,2015-11-15,2017-11-14,MASSACHUSETTS INSTITUTE OF TECHNOLOGY,POSTDOCTORAL ASSOCIATE,15-2041,STATISTICIANS,...,Y,58032.0,58032.0,MA,0.0,northeast,362,98.6,726,730
1,CERTIFIED,2016-05-13,2017-10-01,H-1B,2016-09-01,2019-08-31,MASSACHUSETTS INSTITUTE OF TECHNOLOGY (MIT),ASSISTANT PROFESSOR,25-1031,"ARCHITECTURE TEACHERS, POSTSECONDARY",...,Y,60967.0,95600.0,MA,34633.0,northeast,50,96.0,506,1094
2,CERTIFIED,2016-11-04,2017-10-01,H-1B,2016-12-01,2017-11-30,MASSACHUSETTS INSTITUTE OF TECHNOLOGY,POSTDOCTORAL ASSOCIATE,19-2012,PHYSICISTS,...,Y,50398.0,60400.0,MA,10002.0,northeast,362,98.6,331,364
3,CERTIFIED,2016-12-28,2017-10-01,H-1B,2017-02-01,2018-01-31,MASSACHUSETTS INSTITUTE OF TECHNOLOGY,RESEARCH SCIENTIST,19-2042,"GEOSCIENTISTS, EXCEPT HYDROLOGISTS AND GEOGRAP...",...,Y,61069.0,65000.0,MA,3931.0,northeast,362,98.6,277,364
4,CERTIFIED,2017-02-24,2017-10-01,H-1B,2017-06-01,2018-05-31,MASSACHUSETTS INSTITUTE OF TECHNOLOGY,POSTDOCTORAL ASSOCIATE,17-2131,MATERIALS ENGINEERS,...,Y,41392.0,49383.0,MA,7991.0,northeast,362,98.6,219,364


In [14]:
df.to_csv("datasets/2017_2018_preprocessing3.csv", index=False)